In [1]:
import os
import sys
import math
import numpy as np
import pandas as pd
from itertools import groupby
R = 6.371*10**6

In [2]:
def unique(list1): 
  # intilize a null list 
  unique_list = []    
  # traverse for all elements 
  for x in list1: 
      # check if exists in unique_list or not 
      if x not in unique_list: 
          unique_list.append(x) 
  return unique_list

In [3]:
def cartesian(lat,lon):
  lat = lat/180*math.pi
  lon = lon/180*math.pi
  z = R*np.sin(lat)
  u = R*np.cos(lat)
  x = u*np.cos(lon)
  y = u*np.sin(lon)
  return x,y,z

In [4]:
def great_circle_dist(lat1,lon1,lat2,lon2):
  lat1 = lat1/180*math.pi
  lon1 = lon1/180*math.pi
  lat2 = lat2/180*math.pi
  lon2 = lon2/180*math.pi
  temp = np.cos(lat1)*np.cos(lat2)*np.cos(lon1-lon2)+np.sin(lat1)*np.sin(lat2)
  if temp>1:
    temp=1
  if temp<-1:
    temp=-1
  theta = np.arccos(temp)
  d = theta*R
  return d

In [5]:
def shortest_dist_to_great_circle(lat,lon,lat_start,lon_start,lat_end,lon_end):
  if abs(lat_start-lat_end)<1e-6 and abs(lon_start-lon_end)<1e-6:
    return np.zeros(len(lat))
  else:
    x,y,z = cartesian(lat,lon)
    x_start,y_start,z_start = cartesian(lat_start,lon_start)
    x_end,y_end,z_end = cartesian(lat_end,lon_end)
    cross_product = np.cross(np.array([x_start,y_start,z_start]),np.array([x_end,y_end,z_end]))
    N = cross_product/(np.linalg.norm(cross_product)+1e-6)
    C = np.array([x,y,z])/R
    temp = np.dot(N,C)
    temp[temp>1]=1
    temp[temp<-1]=-1
    NOC = np.arccos(temp)
    d = abs(math.pi/2-NOC)*R
    return d

In [6]:
def pairwise_great_circle_dist(latlon_array):
  dist = []
  k = np.shape(latlon_array)[0]
  for i in range(k-1):
    for j in np.arange(i+1,k):
      dist.append(great_circle_dist(latlon_array[i,0],latlon_array[i,1],latlon_array[j,0],latlon_array[j,1]))
  return dist

In [7]:
def ExistKnot(mat,r,w):
  n = mat.shape[0]
  if n>1:
    lat_start = mat[0,2]
    lon_start = mat[0,3]
    lat_end = mat[n-1,2]
    lon_end = mat[n-1,3]
    lat = mat[:,2]
    lon = mat[:,3]                
    d = shortest_dist_to_great_circle(lat,lon,lat_start,lon_start,lat_end,lon_end)
    if max(d)<w:
      return 0, None
    else:
      return 1, np.argmax(d)
  else:
    return 0, None

In [16]:
def ExtractFlights(mat,itrvl,r,w,h):
  if len(mat.shape)==1:
    out = np.array([3,mat[2],mat[3],mat[1]-itrvl/2,None,None,mat[1]+itrvl/2])
  elif len(mat.shape)==2 and mat.shape[0]==1:
    out = np.array([3,mat[0,2],mat[0,3],mat[0,1]-itrvl/2,None,None,mat[0,1]+itrvl/2])
  else:
    n = mat.shape[0]
    mat = np.hstack((mat,np.arange(n).reshape((n,1))))
    if n>1 and max(pairwise_great_circle_dist(mat[:,2:4]))<r:
      m_lon = (mat[0,2]+mat[n-1,2])/2
      m_lat = (mat[0,3]+mat[n-1,3])/2
      out = np.array([2,m_lon,m_lat,mat[0,1]-itrvl/2,m_lon,m_lat,mat[n-1,1]+itrvl/2])
    else:
      complete = 0
      knots = [0,n-1]
      mov = np.array([great_circle_dist(mat[i,2],mat[i,3],mat[i+1,2],mat[i+1,3]) for i in range(n-1)])
      pause_index = np.arange(0,n-1)[mov<h]
      temp = []
      for j in range(len(pause_index)-1):
        if pause_index[j+1]-pause_index[j]==1:
          temp.append(pause_index[j])
          temp.append(pause_index[j+1]) 
      ## all the consequential numbers in between are inserted twice, but start and end are inserted once
      long_pause = np.unique(temp)[np.array([len(list(group)) for key, group in groupby(temp)])==1]
      ## pause 0,1,2, correspond to point [0,1,2,3], so the end number should plus 1
      long_pause[np.arange(1,len(long_pause),2)] = long_pause[np.arange(1,len(long_pause),2)]+1
      knots.extend(long_pause.tolist())
      knots.sort()
      knots = unique(knots)
      while complete == 0:
        mat_list = []
        for i in range(len(knots)-1):
          mat_list.append(mat[knots[i]:min(knots[i+1]+1,n-1),:])
        knot_yes = np.empty(len(mat_list))
        knot_pos = np.empty(len(mat_list))
        for i in range(len(mat_list)):
          knot_yes[i] , knot_pos[i] = ExistKnot(mat_list[i],r,w)
        if sum(knot_yes)==0:
          complete = 1  
        else:
          for i in range(len(mat_list)):
            if knot_yes[i]==1:
              knots.append(int((mat_list[i])[int(knot_pos[i]),4]))
          knots.sort()
      out = []
      for j in range(len(knots)-1):
        start = knots[j]
        end = knots[j+1]
        mov = np.array([great_circle_dist(mat[i,2],mat[i,3],mat[i+1,2],mat[i+1,3]) for i in np.arange(start,end)])
        if sum(mov>=h)==0:
          m_lon = (mat[start,2]+mat[end,2])/2
          m_lat = (mat[start,3]+mat[end,3])/2
          nextline = [2, m_lon,m_lat,mat[start,1],m_lon,m_lat,mat[end,1]]
        else:
          nextline = [1, mat[start,2],mat[start,3],mat[start,1],mat[end,2],mat[end,3],mat[end,1]]
        out.append(nextline)
      out = np.array(out)
  return out

In [21]:
# filelist:  vector of csv files containing GPS trace
# itrvl:     Interval width (in seconds) that observations are averaged over
# r:         Minimum distance between consecutive locations needed to be
#            covered to be considered movement (instead of a pause)
# w:         Maximum allowed distance from the flight path for intermediate points
# h:         the maximal radius of pause during flights 
# code:     1=flight
#           2=pause
#           3=unclassified
#           4=missing data

path = 'C:/Users/glius/Google Drive/Gaussian Process/gps-iphone6'
filelist = [path + "/"+ os.listdir(path)[j] for j in range(len(os.listdir(path)))]

def GPS2MobMat(filelist,itrvl=10,accuracylim=51, r=None, w=None,h=None):
  if r is None:
    r = np.sqrt(itrvl)
  if h is None:
    h = r
  data = pd.DataFrame()
  sys.stdout.write("Read in all GPS csv files..." + '\n')
  for i in range(len(filelist)):
    df = pd.read_csv(filelist[i])
    data = data.append(df)
  data = data[data.accuracy<accuracylim]  
  if w is None:
    w = np.mean(data.accuracy)+itrvl
  t_start = np.array(data.timestamp)[0]/1000
  t_end = np.array(data.timestamp)[-1]/1000
  avgmat = np.empty([int(np.ceil((t_end-t_start)/itrvl))+2,4])
  sys.stdout.write("Collapse data within " +str(itrvl)+" second intervals..."+'\n')
  IDam = 0
  count = 0
  nextline=[1,t_start+itrvl/2,data.iloc[0,2],data.iloc[0,3]] 
  numitrvl=1
  for i in np.arange(1,data.shape[0]):
    if data.iloc[i,0]/1000 < t_start+itrvl:
      nextline[2]=nextline[2]+data.iloc[i,2]
      nextline[3]=nextline[3]+data.iloc[i,3]
      numitrvl=numitrvl+1
    else:
      nextline[2]=nextline[2]/numitrvl
      nextline[3]=nextline[3]/numitrvl
      avgmat[IDam,:]=nextline
      count=count+1
      IDam=IDam+1
      nummiss=int(np.floor((data.iloc[i,0]/1000-(t_start+itrvl))/itrvl))
      if nummiss>0:
        avgmat[IDam,:] = [4,t_start+itrvl,t_start+itrvl*(nummiss+1),None]
        count=count+1
        IDam=IDam+1
      t_start=t_start+itrvl*(nummiss+1)
      nextline[0]=1
      nextline[1]=t_start+itrvl/2
      nextline[2]=data.iloc[i,2]
      nextline[3]=data.iloc[i,3]
      numitrvl=1

  avgmat = avgmat[0:count,:]
  ID1 = avgmat[:,0]==1
  outmat = np.zeros(7)
  curind = 0
  sys.stdout.write("Extract flights and pauses ..."+'\n')
  for i in range(avgmat.shape[0]):
    if avgmat[i,0]==4:
      #print(curind,i)
      temp = ExtractFlights(avgmat[np.arange(curind,i),:],itrvl,r,w,h)
      outmat = np.vstack((outmat,temp))
      curind=i+1
  if curind<avgmat.shape[0]:
    print(np.arange(curind,avgmat.shape[0]))
    temp = ExtractFlights(avgmat[np.arange(curind,avgmat.shape[0]),:],itrvl,r,w,h)
    outmat = np.vstack((outmat,temp))

  mobmat = np.delete(outmat,0,0)
  #mobmat.columns = ['code','x0','y0','t0','x1','y1','t1']
  return mobmat

In [14]:
## deal with the unclassified, merge pauses after that, and connect flights and pauses
def InferMobMat(mobmat,itrvl=10,r=None):
  ## infer those unclassified pieces
  sys.stdout.write("Infer unclassified windows ..."+'\n')
  if r is None:
    r = np.sqrt(itrvl)
  code = mobmat[:,0]
  x0 = mobmat[:,1]; y0 = mobmat[:,2]; t0 = mobmat[:,3]
  x1 = mobmat[:,4]; y1 = mobmat[:,5]; t1 = mobmat[:,6]
  
  for i in range(len(code)):
    if code[i]==3 and i==0:
      code[i]=2
      x1[i] = x0[i]
      y1[i] = y0[i]
    if code[i]==3 and i>0:
      d = great_circle_dist(x0[i],y0[i],x1[i-1],y1[i-1])
      if t0[i]-t1[i-1]<=itrvl*3:
        if d<r:
          code[i]=2
          x1[i] = x0[i]
          y1[i] = y0[i]
        else:
          code[i]=1
          s_x = x0[i]-itrvl/2/(t0[i]-t1[i-1])*(x0[i]-x1[i-1])
          s_y = y0[i]-itrvl/2/(t0[i]-t1[i-1])*(y0[i]-y1[i-1])
          e_x = x0[i]+itrvl/2/(t0[i]-t1[i-1])*(x0[i]-x1[i-1])
          e_y = y0[i]+itrvl/2/(t0[i]-t1[i-1])*(y0[i]-y1[i-1])
          x0[i] = s_x; x1[i]=e_x
          y0[i] = s_y; y1[i]=e_y
      if t0[i]-t1[i-1]>itrvl*3:
        if (i+1)<len(code):
          f = great_circle_dist(x0[i],y0[i],x0[i+1],y0[i+1])
          if t0[i+1]-t1[i]<=itrvl*3:
            if f<r:
              code[i]=2
              x1[i] = x0[i]
              y1[i] = y0[i]
            else:
              code[i]=1
              s_x = x0[i]-itrvl/2/(t0[i+1]-t1[i])*(x0[i+1]-x0[i])
              s_y = y0[i]-itrvl/2/(t0[i+1]-t1[i])*(y0[i+1]-y0[i])
              e_x = x0[i]+itrvl/2/(t0[i+1]-t1[i])*(x0[i+1]-x0[i])
              e_y = y0[i]+itrvl/2/(t0[i+1]-t1[i])*(y0[i+1]-y0[i])
              x0[i] = s_x; x1[i]=e_x
              y0[i] = s_y; y1[i]=e_y
          else:
            code[i]=2
            x1[i] = x0[i]
            y1[i] = y0[i]
        else:
          code[i]=2
          x1[i] = x0[i]
          y1[i] = y0[i]
    mobmat[i,:] = [code[i],x0[i],y0[i],t0[i],x1[i],y1[i],t1[i]]
  
  ## merge consecutive pauses
  sys.stdout.write("Merge consecutive pauses and bridge gaps ..."+'\n')
  k = []
  for j in np.arange(1,len(code)):
    if code[j]==2 and code[j-1]==2 and t0[j]==t1[j-1]:
      k.append(j-1)
      k.append(j)
  ## all the consequential numbers in between are inserted twice, but start and end are inserted once
  rk = np.unique(k)[np.array([len(list(group)) for key, group in groupby(k)])==1]  
  for j in range(int(len(rk)/2)):
    start = rk[2*j]
    end = rk[2*j+1]
    mx = np.mean(x0[np.arange(start,end+1)])
    my = np.mean(y0[np.arange(start,end+1)])
    mobmat[start,:] = [2,mx,my,t0[start],mx,my,t1[end]]
    mobmat[np.arange(start+1,end+1),0]=5 
  mobmat = mobmat[mobmat[:,0]!=5,:]
  
  ## check missing intervals, if starting and ending point are close, make them same
  for j in np.arange(1,mobmat.shape[0]):
    if mobmat[j,3] > mobmat[j-1,6]:
      d = great_circle_dist(mobmat[j,1],mobmat[j,2],mobmat[j-1,4],mobmat[j-1,5])
      if d<5:
        if mobmat[j,0]==2 and mobmat[j-1,0]==2:
          mean_x = (mobmat[j,1] + mobmat[j-1,4])/2
          mean_y = (mobmat[j,2] + mobmat[j-1,5])/2
          mobmat[j,1] = mobmat[j,4] = mobmat[j-1,1] = mobmat[j-1,4] = mean_x
          mobmat[j,2] = mobmat[j,5] = mobmat[j-1,2] = mobmat[j-1,5] = mean_y
        if mobmat[j,0]==1 and mobmat[j-1,0]==2:
          mobmat[j,1] = mobmat[j-1,4]
          mobmat[j,2] = mobmat[j-1,5]
        if mobmat[j,0]==2 and mobmat[j-1,0]==1:
          mobmat[j-1,4] = mobmat[j,1]
          mobmat[j-1,5] = mobmat[j,2]
        if mobmat[j,0]==1 and mobmat[j-1,0]==1:
          mean_x = (mobmat[j,1] + mobmat[j-1,4])/2
          mean_y = (mobmat[j,2] + mobmat[j-1,5])/2
          mobmat[j-1,4] = mobmat[j,1] = mean_x
          mobmat[j-1,5] = mobmat[j,2] = mean_y

  ## connect flights and pauses
  for j in np.arange(1,mobmat.shape[0]):
    if mobmat[j,0]*mobmat[j-1,0]==2 and mobmat[j,3]==mobmat[j-1,6]:
      if mobmat[j,0]==1:
        mobmat[j,1] = mobmat[j-1,4]
        mobmat[j,2] = mobmat[j-1,5]
      if mobmat[j-1,0]==1:
        mobmat[j-1,4] = mobmat[j,1]
        mobmat[j-1,5] = mobmat[j,2] 
  return mobmat

In [22]:
path = 'C:/Users/glius/Google Drive/Gaussian Process/gps-iphone6'
filelist = [path + "/"+ os.listdir(path)[j] for j in range(len(os.listdir(path)))]
a = GPS2MobMat(filelist,itrvl=10,accuracylim=51, r=1, w=None,h=None)

Read in all GPS csv files...
Collapse data within 10 second intervals...
Extract flights and pauses ...


In [23]:
b = InferMobMat(a,itrvl=10,r=None)

Infer unclassified windows ...
Merge consecutive pauses and bridge gaps ...


In [24]:
np.save("MobMat",b)